In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In this notebook we are diving deeper in the data preparation step of out workflow, here is an overview of what data preparation entails

1. Splitting data into features and labels (x and y)
2. Converting non-numerical values to numerical values (encoding)
3. Dealing with missing values, either by inputing our removing rows/columns
4. Splitting data into training and test sets

In [2]:
heart_disease = pd.read_csv("resources/heart-disease.csv")
heart_disease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
# Splitting features and labels is as easy as dropping the label column and then creating a separate series for it
x = heart_disease.drop("target", axis=1)
y = heart_disease["target"]

In [4]:
# Step 4, splitting training and test data is also easy if we use sklearn
from sklearn.model_selection import train_test_split
# We can use the test_size parameter to specify how much of our data is going to be used for testing (20% here)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

That leaves us with steps 2 and 3, which are the hardest and most important ones, let's import data with missing values

In [15]:
car_sales = pd.read_csv("resources/car-sales-extended.csv")
car_sales.sample(5)

,Make,Colour,Odometer (KM),Doors,Price
732,Honda,White,121791,4,6920
138,Toyota,Red,133195,4,14795
804,Toyota,White,185146,4,4566
545,Honda,White,175309,4,4034
393,Toyota,Blue,164956,4,10834


In [16]:
# We have missing values, let's get a look at how many values we are missing
car_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Make           1000 non-null   object
 1   Colour         1000 non-null   object
 2   Odometer (KM)  1000 non-null   int64 
 3   Doors          1000 non-null   int64 
 4   Price          1000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 39.2+ KB


In [17]:
# Let's deal with our categorical features by using sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [39]:
"""
We define doors as a categorical feature here because
having more doors doesn't necessarily implies in a higher price and since
most models have 1 coeficient for each feature, this could harms our performance later.
Think of a linear model to predict the price, it could be something like y(x) = 50*doors + ...
which would mean that each dooor adds 50 to the final price.
Now, if we OHE the model could be something like y(x) = 30 * has_3_doors + 50 * has_4_doors +10 * has_5_doors
which can capture the fact that people may not like cars with 5 or 3 doors that much
"""
categorical_features = ["Make", "Colour", "Doors"]
"""
A One Hot Encoder takes a categorical feature (such as Make) and encodes it in n different values
that are either 0 or 1 where n is the number of categories, so if we have 3 makes we will get
3 new columns, one for each make, and only the column that matches that car's make will be 1.
Only one of the columns will be "hot" and thus One Hot encoding
"""
ohe = OneHotEncoder()
"""
The Column tranformers receives and operation to apply and a list of columns to apply that operation
to and applies it, then it will passthrough the remaining columns (there are other options for this)
"""
transformer = ColumnTransformer([("one-hot", ohe, categorical_features)], remainder='passthrough')

transformed_data = pd.DataFrame(transformer.fit_transform(car_sales))
transformed_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,35431.0,15323.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0,19943.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,84714.0,28343.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,154365.0,13434.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0,14043.0


## Dealing with missing values

In [41]:
cars_missing = pd.read_csv("resources/car-sales-extended-missing-data.csv")
cars_missing.sample(5)

,Make,Colour,Odometer (KM),Doors,Price
995,Toyota,Black,35820.0,4.0,32042.0
88,Nissan,Red,61892.0,NaN,18160.0
585,Honda,Blue,176566.0,4.0,13257.0
289,Nissan,White,215130.0,4.0,11768.0
915,BMW,Blue,152772.0,3.0,14588.0


There are two ways to deal with missing data: imputing and removing.
Inputing is about generating values to fill the missing data you have, while removing just drop columns or rows with missing data altogether

In [43]:
# Let's just see how manyu missing values we have in each column
cars_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [46]:
# We can fill missing data with pandas by using .fillna()
cars_missing["Make"].fillna("Missing", inplace=True)
cars_missing["Colour"].fillna("Missing", inplace=True)
cars_missing["Odometer (KM)"].fillna(cars_missing["Odometer (KM)"].mean(), inplace=True)
cars_missing["Doors"].fillna(4, inplace=True)

# Remove Rows with missing price since it's our label column
cars_missing.dropna(inplace=True)

In [47]:
cars_missing.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

In [48]:
# We can also handle missing values using sklearn
cars_missing = pd.read_csv("resources/car-sales-extended-missing-data.csv")
cars_missing.sample(5)

,Make,Colour,Odometer (KM),Doors,Price
884,BMW,White,152410.0,5.0,21580.0
705,Toyota,Red,98854.0,4.0,29876.0
306,Honda,Red,108681.0,4.0,10740.0
381,Toyota,Blue,243255.0,4.0,7378.0
702,Honda,White,82518.0,4.0,22587.0


In [62]:
from sklearn.impute import SimpleImputer

categorical_imputer = SimpleImputer(strategy="constant", fill_value="Missing")
door_imputer = SimpleImputer(strategy="constant", fill_value=4)
numerical_imputer = SimpleImputer(strategy="mean")

""" 
If we pass an estimator to the remainder parameter, any column that doesn't fall into one of the
other ones will be tranformed according to it
Note: and estimator is any class with fit, transform, fit_tranform and so on methods
"""
transformer = ColumnTransformer([
                                ("Categorical Imputer", categorical_imputer, ["Make", "Colour"]),
                                ("Door Imputer", door_imputer, ["Doors"])
                                ],
                                remainder=numerical_imputer,)

transformed = pd.DataFrame(transformer.fit_transform(cars_missing))
transformed.columns = ["Make", "Colour", "Doors", "Odometer (KM)", "Price"]
transformed.sample(5)

,Make,Colour,Doors,Odometer (KM),Price
648,Honda,White,4,40134,16273
782,Nissan,Missing,4,219137,5865
611,Toyota,Black,4,136271,20242
145,Nissan,White,4,123923,14986
197,Toyota,Green,4,146940,25133
